In [1]:
import json, random, re, collections, itertools, base64, sys
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
ls = lambda path: print("\n".join(map(str, path.iterdir())))

sys.path.append('../src')
mturk_path = Path("../mturk/results")

%load_ext autoreload
%autoreload 2

In [2]:
from instruction_parsing import program, Commands
import annotation_io
import read_data
annotation_io.set_recipe_source("npn-cooking")
from edit_distance import edit_distance


def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

def annotator_agreement(annotation1,annotation2, ignore_commands=[]):
    lst1 = [(x.command.name,x.arg,x.resource) for x in program(annotation1) if x.command not in ignore_commands]
    lst2 = [(x.command.name,x.arg,x.resource) for x in program(annotation2) if x.command not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))


In [3]:
ls(mturk_path)

../mturk/results/4335716.json
../mturk/results/Batch_4430799_batch_results.csv
../mturk/results/Batch_4430799_batch_results.csv:Zone.Identifier
../mturk/results/results_approved.csv
../mturk/results/4289268.json
../mturk/results/Batch_4258572_batch_results.csv
../mturk/results/Batch_4286064_batch_results.csv
../mturk/results/Batch_4301520_batch_results.csv
../mturk/results/Batch_4289268_batch_results.csv
../mturk/results/Batch_4280306_batch_results.csv
../mturk/results/4286064.json
../mturk/results/4301520.json
../mturk/results/Batch_4335716_batch_results.csv


In [16]:
def load_results(batch):
    df = pd.read_csv(str(next(mturk_path.rglob(f"*{batch}*.csv"))))
    #return df
    #hack:
    df = df.rename(columns={"Input.v11":"Answer.Title", "Input.v3": "Answer.ID"})
    df = df[[c for c in df.columns if "Answer." in c or 'Id' in c]]
    df = df.rename(columns={c:c.split('.',1)[1] for c in df.columns if '.' in c})
    df = df.drop("id", axis=1).rename(columns={"ID": "id"})
    return df.set_index("AssignmentId")
batch = 4430799
df=load_results(batch)
df

,HITId,HITTypeId,WorkerId,id,Title,actions,ingredients,seconds_spent
AssignmentId,,,,,,,,
3DZQRBDBSRAR4M5SWQ7ZVSOZVJ6S3S,3RWO3EJEMM4WU7ERR3KLNA8403T1PQ,325VGVP4D8XQMOMUXKSUD492QGN0LP,ASCLZFXD9WMON,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IRghrxgvV...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...","[407,207,275,191,84,0,21,223,246]"
3VNL7UK1XLENZYY05LDR669I0S4TF6,3RWO3EJEMM4WU7ERR3KLNA8403T1PQ,325VGVP4D8XQMOMUXKSUD492QGN0LP,A1NLXD111QHDKZ,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""1"",""ts"":1,""arg"":""InaG8xAmv...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...","[84,9,8,2,1,0,0,0,0]"
3YJ6NA41JHB5C8RP7UVSGU0D4W6JP0,3RWO3EJEMM4WU7ERR3KLNA8403T1PQ,325VGVP4D8XQMOMUXKSUD492QGN0LP,A2A9O97AGBORB4,cuban_red_beans__rice,Cuban Red Beans Rice,"[{""id"":0,""command"":""3"",""ts"":1,""arg"":""TSHEET"",""...","[{""name"":""bay"",""id"":""Ix38RUchf""},{""name"":""bean...","[24,273,1,0,1,0,0,0,0]"
382M9COHENAATXDTAFJVTKAK75WEU0,3YKP7CX6H7A81ZSJXEJTT0USMLEB7C,325VGVP4D8XQMOMUXKSUD492QGN0LP,AQG3UUVNSKHJR,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,[],"[{""name"":""pepper"",""id"":""Is_lS1dmt""},{""name"":""s...","[178,11,0,0,0,1,0,0,1]"
3LRLIPTPEW4T0OR50WD7REBVKZJKAX,3YKP7CX6H7A81ZSJXEJTT0USMLEB7C,325VGVP4D8XQMOMUXKSUD492QGN0LP,A378DVZDTWCGQ7,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":0,""command"":""4"",""ts"":1,""arg"":""LBOIL"",""a...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""ch...","[387,6,3,5,0,1,0,1,1]"
3PMBY0YE2DYXC75CGDI970N398SC9X,3YKP7CX6H7A81ZSJXEJTT0USMLEB7C,325VGVP4D8XQMOMUXKSUD492QGN0LP,A3BCDYK5ZHQ563,augustas_chilled_tomato_soup_basil_cream,Augustas Chilled Tomato Soup Basil Cream,"[{""id"":0,""command"":""0"",""ts"":1,""arg"":""IdCkp3LYx...","[{""name"":""basil"",""id"":""IaGLvtAKO""},{""name"":""ch...","[298,16,3,1,1,1,0,1,2]"
3EO896NRA2Q340KWOB4CL41ATZYTJM,32W3UF2E0TGUSH269GHH18G0D4O4CS,325VGVP4D8XQMOMUXKSUD492QGN0LP,AWJ3MAVSHICCR,senators_navy_bean_soup,Senators Navy Bean Soup,"[{""id"":0,""command"":""2"",""ts"":1,""arg"":""TLID"",""ar...","[{""name"":""bean"",""id"":""IRghrxgvV""},{""name"":""car...","[42,64,29,37,53,36]"
3GNA64GUZKZI5LDJI56NFP7IRR5Q5K,32W3UF2E0TGUSH269GHH18G0D4O4CS,325VGVP4D8XQMOMUXKSUD492QGN0LP,A3FE33S2NMC46Q,senators_navy_bean_soup,Senators Navy Bean Soup,[],"[{""name"":""bean"",""id"":""IRghrxgvV""},{""name"":""car...","[18,1,0,0,1,0]"
3WSELTNVR9XS37HSU5RZS5YR3CXATG,32W3UF2E0TGUSH269GHH18G0D4O4CS,325VGVP4D8XQMOMUXKSUD492QGN0LP,A1EVYY9TPBYTYB,senators_navy_bean_soup,Senators Navy Bean Soup,"[{""id"":0,""command"":""2"",""ts"":1,""arg"":""TPOT"",""ar...","[{""name"":""bean"",""id"":""IRghrxgvV""},{""name"":""car...","[3,230,191,79,181,26]"


In [25]:
def actions2program(actions):
    cmds = ["PUT", "REMOVE", "USE", "STOP_USING","CHEF_CHECK","DO","MOVE_CONTENTS"]
    prog = []
    if type(actions)==str:
        actions = json.loads(actions)
    for a in actions:
        prog.append((int(a["ts"]), cmds[int(a["command"])], a["arg"], a["resource"]))
    prog = sorted(prog)
    prog = [c[1:] for c in prog]
    return prog
df["program"] = df["actions"].apply(actions2program)
df["n"] = df["program"].apply(len)
df[["WorkerId","id", "n"]]

,WorkerId,id,n
AssignmentId,,,
3DZQRBDBSRAR4M5SWQ7ZVSOZVJ6S3S,ASCLZFXD9WMON,cuban_red_beans__rice,16
3VNL7UK1XLENZYY05LDR669I0S4TF6,A1NLXD111QHDKZ,cuban_red_beans__rice,12
3YJ6NA41JHB5C8RP7UVSGU0D4W6JP0,A2A9O97AGBORB4,cuban_red_beans__rice,19
382M9COHENAATXDTAFJVTKAK75WEU0,AQG3UUVNSKHJR,augustas_chilled_tomato_soup_basil_cream,0
3LRLIPTPEW4T0OR50WD7REBVKZJKAX,A378DVZDTWCGQ7,augustas_chilled_tomato_soup_basil_cream,7
3PMBY0YE2DYXC75CGDI970N398SC9X,A3BCDYK5ZHQ563,augustas_chilled_tomato_soup_basil_cream,10
3EO896NRA2Q340KWOB4CL41ATZYTJM,AWJ3MAVSHICCR,senators_navy_bean_soup,15
3GNA64GUZKZI5LDJI56NFP7IRR5Q5K,A3FE33S2NMC46Q,senators_navy_bean_soup,0
3WSELTNVR9XS37HSU5RZS5YR3CXATG,A1EVYY9TPBYTYB,senators_navy_bean_soup,12


In [27]:
def annotator_agreement(lst1,lst2, ignore_commands=[]):
    lst1 = [x for x in lst1 if x[0] not in ignore_commands]
    lst2 = [x for x in lst2 if x[0] not in ignore_commands]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))

In [32]:
def couple_agreement(lst):
    ret=[]
    for l1, l2 in itertools.combinations(lst,2):
        ret.append(annotator_agreement(l1,l2))
    return ret
df_agreement=df[df["n"]>0].groupby("id")["program"].apply(list).apply(couple_agreement)
df_agreement

id
augustas_chilled_tomato_soup_basil_cream                                 [0.4117647058823529]
baked_chicken_tomato_sauce                   [0.125, 0.10526315789473684, 0.4516129032258065]
beef_bean_chili                               [0.5, 0.33333333333333337, 0.33333333333333337]
braised_celery__red_pepper                  [0.32352941176470584, 0.38888888888888884, 0.2...
broccoli_salad_bacon                        [0.23529411764705888, 0.2857142857142857, 0.47...
carrot_orange_soup                                                                         []
contadina_garden_sauce                      [0.29166666666666663, 0.3928571428571429, 0.38...
cuban_red_beans__rice                       [0.4285714285714286, 0.4571428571428572, 0.387...
fast_pilaf                                                 [0.46153846153846156, 0.4, 0.4375]
fourway-chicken                                                          [0.8260869565217391]
garlic_bread_in_a_bag                                    

In [35]:
df_agreement.apply(np.mean)

/home/ugoren/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


id
augustas_chilled_tomato_soup_basil_cream    0.411765
baked_chicken_tomato_sauce                  0.227292
beef_bean_chili                             0.388889
braised_celery__red_pepper                  0.315251
broccoli_salad_bacon                        0.333090
carrot_orange_soup                               NaN
contadina_garden_sauce                      0.357804
cuban_red_beans__rice                       0.424270
fast_pilaf                                  0.433013
fourway-chicken                             0.826087
garlic_bread_in_a_bag                       0.200000
glazed_veggies_on_sticks                    0.290252
homemade_croutons                           0.220798
honey_roast_carrots                         0.529412
kohls_quick_hot_n_sweet_sauce               0.500000
parmesan_risotto                            0.465783
potatoes_primavera                          0.352941
senators_navy_bean_soup                     0.444444
sweet__sour_chicken_wings                  